In [38]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import minmax_scale, OneHotEncoder

import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

with open('randomized_data.txt','r') as file:
    data=file.readlines()

targets=[]
    
for i_line, line in enumerate(data):
    data_line = line.split(',')
    data_line[13] = (data_line[13])[:-1]
    data_line=[float(st) for st in data_line] 
    targets.append(data_line.pop(0))
    data[i_line]=data_line

predict_data=np.array([[13.72, 1.43, 2.5, 16.7, 108, 3.4, 3.67, 0.19, 2.04, 6.8, 0.89, 2.87, 1285],
                       [12.04, 4.3, 2.38, 22, 80, 2.1, 1.75, 0.42, 1.35, 2.6, 0.79, 2.57, 580],
                       [14.13, 4.1, 2.74, 24.5, 96, 2.05, 0.76, 0.56, 1.35, 9.2, 0.61, 1.6, 560]])


data=np.append(data, predict_data, axis=0)

data=minmax_scale(np.array(data), copy=False)
targets=np.array(targets)
predict_data=data[len(data)-3:]
data=data[:-3]

#Splitting data into 75% train and 25% test
train_x, test_x, train_y, test_y = train_test_split(data, targets, test_size=0.25)

encoder = OneHotEncoder(sparse=False)
train_y_enc = encoder.fit_transform(train_y.reshape(-1,1))
test_y_enc = encoder.fit_transform(test_y.reshape(-1,1))

model = Sequential()
model.add(Dense(5, input_shape=(13,), activation='sigmoid', name='hiddeninput'))
model.add(Dense(3, activation='softmax', name='output'))
model.compile(Adam(lr=0.1), loss='categorical_crossentropy', metrics=['accuracy'])
history=model.fit(train_x, train_y_enc, epochs=50, verbose=0)

ynew = model.predict(predict_data)
print(np.round(ynew))


[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [20]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import minmax_scale, OneHotEncoder

import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

#Open data file
with open('randomized_data.txt','r') as file:
    data=file.readlines()

targets=[]

#Build data into proper format
for i_line, line in enumerate(data):
    data_line = line.split(',')
    data_line[13] = (data_line[13])[:-1]
    data_line=[float(st) for st in data_line] 
    targets.append(data_line.pop(0))
    data[i_line]=data_line

#Scale data between 0 and 1
data=minmax_scale(np.array(data), copy=False)
targets=np.array(targets)

#Splitting data into 75% train and 25% test
train_x, test_x, train_y, test_y = train_test_split(data, targets, test_size=0.25)

#One hot encode the target data
encoder = OneHotEncoder(sparse=False)
train_y_enc = encoder.fit_transform(train_y.reshape(-1,1))
test_y_enc = encoder.fit_transform(test_y.reshape(-1,1))

num_nodes=[1,2,5,1000]

#Empty arrays for final training/validation errors (avg of shuffle errors)
final_train_error=np.zeros((4,4))
final_valid_error=np.zeros((4,4))

#Making early stop call back 10 epochs after validation loss starts to increase
#This is fine because the later code takes the minimum total error
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)

#For each number of nodes
for i_nodes, nodes in enumerate(num_nodes):
    #Specify number of folds
    kfold = KFold(n_splits=10, shuffle=True)
    #For each number of layers
    for num_layers in range(4):
        #Build model
        model = Sequential()
        model.add(Dense(nodes, input_shape=(13,), activation='sigmoid', name='hiddeninput'))
        #Add layers
        for layers in range(num_layers):
            model.add(Dense(nodes, input_shape=(13,), activation='sigmoid'))
    
        model.add(Dense(3, activation='softmax', name='output'))
        model.compile(Adam(lr=0.1), loss='categorical_crossentropy', metrics=['accuracy'])
        
        train_shuffle_errors=np.array([])
        valid_shuffle_errors=np.array([])
        #Shuffle 5 times
        for i_shuffle in range(5):
            #Empty arrays to hold each k-cross error
            train_kcross_errors=np.array([])
            valid_kcross_errors=np.array([])
            #Perform k-cross validation
            for train, evalu in kfold.split(train_x, train_y_enc):
                #Training model with callback to early stop
                history=model.fit(train_x[train], train_y_enc[train], validation_data=(train_x[evalu], train_y_enc[evalu]), epochs=50, verbose=0, callbacks=[es])
                #Finding index when total error is the smallest
                total_error=np.array(history.history['val_loss'])+np.array(history.history['loss'])
                i_valid_min=np.argmin(total_error)
                #Add training/validation errors when total error is smallest
                train_kcross_errors=np.append(train_kcross_errors, np.array(history.history['loss'])[i_valid_min])
                valid_kcross_errors=np.append(valid_kcross_errors, np.array(history.history['val_loss'])[i_valid_min])
            #Add averaged k-cross errors to shuffle array
            train_shuffle_errors=np.append(train_shuffle_errors, np.sum(train_kcross_errors)/10)
            valid_shuffle_errors=np.append(valid_shuffle_errors, np.sum(valid_kcross_errors)/10)
        #Add averaged shuffle errors to final array
        final_train_error[i_nodes][num_layers]=np.sum(train_shuffle_errors)/5
        final_valid_error[i_nodes][num_layers]=np.sum(valid_shuffle_errors)/5

print(final_train_error)
print(final_valid_error)
                

Epoch 00037: early stopping
Epoch 00043: early stopping
Epoch 00012: early stopping
Epoch 00011: early stopping
Epoch 00011: early stopping
Epoch 00011: early stopping
Epoch 00012: early stopping
Epoch 00011: early stopping
Epoch 00015: early stopping
Epoch 00015: early stopping
Epoch 00012: early stopping
Epoch 00012: early stopping
Epoch 00012: early stopping
Epoch 00026: early stopping
Epoch 00011: early stopping
Epoch 00014: early stopping
Epoch 00011: early stopping
Epoch 00011: early stopping
Epoch 00012: early stopping
Epoch 00011: early stopping
Epoch 00014: early stopping
Epoch 00011: early stopping
Epoch 00018: early stopping
Epoch 00012: early stopping
Epoch 00011: early stopping
Epoch 00018: early stopping
Epoch 00011: early stopping
Epoch 00015: early stopping
Epoch 00015: early stopping
Epoch 00016: early stopping
Epoch 00011: early stopping
Epoch 00013: early stopping
Epoch 00011: early stopping
Epoch 00011: early stopping
Epoch 00018: early stopping
Epoch 00011: early s

Epoch 00038: early stopping
Epoch 00014: early stopping
Epoch 00022: early stopping
Epoch 00017: early stopping
Epoch 00011: early stopping
Epoch 00012: early stopping
Epoch 00012: early stopping
Epoch 00012: early stopping
Epoch 00023: early stopping
Epoch 00015: early stopping
Epoch 00012: early stopping
Epoch 00011: early stopping
Epoch 00012: early stopping
Epoch 00013: early stopping
Epoch 00012: early stopping
Epoch 00015: early stopping
Epoch 00012: early stopping
Epoch 00020: early stopping
Epoch 00012: early stopping
Epoch 00011: early stopping
Epoch 00011: early stopping
Epoch 00013: early stopping
Epoch 00012: early stopping
Epoch 00011: early stopping
Epoch 00012: early stopping
Epoch 00017: early stopping
Epoch 00012: early stopping
Epoch 00011: early stopping
Epoch 00014: early stopping
Epoch 00025: early stopping
Epoch 00017: early stopping
Epoch 00012: early stopping
Epoch 00021: early stopping
Epoch 00012: early stopping
Epoch 00012: early stopping
Epoch 00017: early s

Epoch 00044: early stopping
Epoch 00047: early stopping
Epoch 00013: early stopping
Epoch 00016: early stopping
Epoch 00029: early stopping
Epoch 00036: early stopping
Epoch 00015: early stopping
Epoch 00011: early stopping
Epoch 00018: early stopping
Epoch 00013: early stopping
Epoch 00049: early stopping
Epoch 00011: early stopping
Epoch 00013: early stopping
Epoch 00014: early stopping
Epoch 00012: early stopping
Epoch 00022: early stopping
Epoch 00020: early stopping
Epoch 00027: early stopping
Epoch 00021: early stopping
Epoch 00013: early stopping
Epoch 00028: early stopping
Epoch 00011: early stopping
Epoch 00016: early stopping
Epoch 00011: early stopping
Epoch 00030: early stopping
Epoch 00020: early stopping
Epoch 00029: early stopping
Epoch 00022: early stopping
Epoch 00011: early stopping
Epoch 00012: early stopping
Epoch 00016: early stopping
Epoch 00021: early stopping
Epoch 00013: early stopping
Epoch 00028: early stopping
Epoch 00017: early stopping
Epoch 00035: early s

Epoch 00013: early stopping
Epoch 00029: early stopping
Epoch 00011: early stopping
Epoch 00041: early stopping
Epoch 00046: early stopping
Epoch 00014: early stopping
Epoch 00019: early stopping
Epoch 00039: early stopping
Epoch 00020: early stopping
Epoch 00015: early stopping
Epoch 00013: early stopping
Epoch 00017: early stopping
Epoch 00012: early stopping
Epoch 00016: early stopping
Epoch 00036: early stopping
Epoch 00014: early stopping
Epoch 00013: early stopping
Epoch 00015: early stopping
Epoch 00015: early stopping
Epoch 00020: early stopping
Epoch 00029: early stopping
Epoch 00011: early stopping
Epoch 00024: early stopping
Epoch 00026: early stopping
Epoch 00023: early stopping
Epoch 00018: early stopping
Epoch 00013: early stopping
Epoch 00028: early stopping
Epoch 00022: early stopping
Epoch 00016: early stopping
Epoch 00015: early stopping
Epoch 00011: early stopping
Epoch 00020: early stopping
Epoch 00019: early stopping
Epoch 00020: early stopping
Epoch 00015: early s

Epoch 00050: early stopping
Epoch 00011: early stopping
Epoch 00035: early stopping
Epoch 00011: early stopping
Epoch 00011: early stopping
Epoch 00011: early stopping
Epoch 00011: early stopping
Epoch 00030: early stopping
Epoch 00022: early stopping
Epoch 00011: early stopping
Epoch 00011: early stopping
Epoch 00013: early stopping
Epoch 00011: early stopping
Epoch 00028: early stopping
Epoch 00014: early stopping
Epoch 00011: early stopping
Epoch 00013: early stopping
Epoch 00017: early stopping
Epoch 00012: early stopping
Epoch 00018: early stopping
Epoch 00011: early stopping
Epoch 00013: early stopping
Epoch 00011: early stopping
Epoch 00011: early stopping
Epoch 00011: early stopping
Epoch 00011: early stopping
Epoch 00011: early stopping
Epoch 00012: early stopping
Epoch 00011: early stopping
Epoch 00011: early stopping
Epoch 00011: early stopping
Epoch 00018: early stopping
Epoch 00012: early stopping
Epoch 00021: early stopping
Epoch 00011: early stopping
Epoch 00020: early s

...

Epoch 00035: early stopping
Epoch 00017: early stopping
Epoch 00011: early stopping
Epoch 00021: early stopping
Epoch 00023: early stopping
Epoch 00032: early stopping
Epoch 00027: early stopping
Epoch 00022: early stopping
Epoch 00020: early stopping
Epoch 00012: early stopping
Epoch 00021: early stopping
Epoch 00027: early stopping
Epoch 00011: early stopping


Epoch 00012: early stopping
Epoch 00011: early stopping
Epoch 00035: early stopping
Epoch 00011: early stopping
Epoch 00011: early stopping
Epoch 00019: early stopping
Epoch 00011: early stopping
Epoch 00012: early stopping
Epoch 00012: early stopping
Epoch 00013: early stopping
Epoch 00011: early stopping
Epoch 00017: early stopping
Epoch 00011: early stopping
Epoch 00011: early stopping
Epoch 00011: early stopping
Epoch 00011: early stopping
Epoch 00011: early stopping
Epoch 00011: early stopping
Epoch 00011: early stopping
Epoch 00011: early stopping
Epoch 00020: early stopping
Epoch 00011: early stopping
Epoch 00011: early stopping
Epoch 00012: early stopping
Epoch 00011: early stopping
Epoch 00017: early stopping
Epoch 00011: early stopping
Epoch 00011: early stopping
Epoch 00011: early stopping
Epoch 00011: early stopping
Epoch 00011: early stopping
Epoch 00011: early stopping


Epoch 00032: early stopping
Epoch 00017: early stopping
Epoch 00019: early stopping
Epoch 00017: early stopping
Epoch 00023: early stopping
Epoch 00018: early stopping
Epoch 00023: early stopping
Epoch 00021: early stopping
Epoch 00017: early stopping
Epoch 00012: early stopping
Epoch 00016: early stopping
Epoch 00020: early stopping
Epoch 00018: early stopping
Epoch 00014: early stopping
Epoch 00014: early stopping
Epoch 00025: early stopping
Epoch 00017: early stopping
Epoch 00022: early stopping
Epoch 00034: early stopping
Epoch 00020: early stopping
Epoch 00011: early stopping
Epoch 00012: early stopping
Epoch 00013: early stopping
Epoch 00028: early stopping
Epoch 00015: early stopping
Epoch 00019: early stopping
Epoch 00017: early stopping
Epoch 00015: early stopping
Epoch 00013: early stopping
Epoch 00029: early stopping
Epoch 00011: early stopping
Epoch 00018: early stopping
Epoch 00011: early stopping
Epoch 00020: early stopping
Epoch 00024: early stopping
Epoch 00013: early s

Epoch 00034: early stopping
Epoch 00018: early stopping
Epoch 00022: early stopping
Epoch 00021: early stopping
Epoch 00012: early stopping
Epoch 00011: early stopping
Epoch 00012: early stopping
Epoch 00031: early stopping
Epoch 00020: early stopping
Epoch 00013: early stopping
Epoch 00017: early stopping
Epoch 00021: early stopping
Epoch 00015: early stopping
Epoch 00013: early stopping
Epoch 00012: early stopping
Epoch 00030: early stopping
Epoch 00013: early stopping
Epoch 00022: early stopping
Epoch 00026: early stopping
Epoch 00031: early stopping
Epoch 00011: early stopping
Epoch 00016: early stopping
Epoch 00012: early stopping
Epoch 00014: early stopping
Epoch 00014: early stopping
Epoch 00025: early stopping
Epoch 00011: early stopping
Epoch 00015: early stopping
Epoch 00014: early stopping
Epoch 00027: early stopping
Epoch 00018: early stopping
Epoch 00017: early stopping
Epoch 00023: early stopping
Epoch 00015: early stopping
Epoch 00013: early stopping
Epoch 00025: early s

Epoch 00027: early stopping
Epoch 00027: early stopping
Epoch 00015: early stopping
Epoch 00013: early stopping
Epoch 00015: early stopping
Epoch 00016: early stopping
Epoch 00017: early stopping
Epoch 00018: early stopping
Epoch 00013: early stopping
Epoch 00015: early stopping
Epoch 00011: early stopping
Epoch 00014: early stopping
Epoch 00017: early stopping
Epoch 00015: early stopping
Epoch 00011: early stopping
Epoch 00012: early stopping
Epoch 00023: early stopping
Epoch 00013: early stopping
Epoch 00012: early stopping
Epoch 00027: early stopping
Epoch 00022: early stopping
Epoch 00025: early stopping
Epoch 00017: early stopping
Epoch 00018: early stopping
Epoch 00019: early stopping
Epoch 00019: early stopping
Epoch 00037: early stopping
Epoch 00011: early stopping
Epoch 00024: early stopping
Epoch 00011: early stopping
Epoch 00013: early stopping
Epoch 00012: early stopping
Epoch 00011: early stopping
Epoch 00022: early stopping
Epoch 00011: early stopping
Epoch 00020: early s